In [1]:
import os
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

from itertools import chain

nb_dir = os.path.split(os.path.split(os.getcwd())[0])[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    sys.path.append(nb_dir + '/path_rnn_v2')
    
from path_rnn_v2.util.embeddings import RandomEmbeddings

In [2]:
pos = pd.read_json('./../../chains_of_reasoning_data/_people_person_nationality/parsed/positive_matrix.json')
neg = pd.read_json('./../../chains_of_reasoning_data/_people_person_nationality/parsed/negative_matrix.json')
dev = pd.read_json('./../../chains_of_reasoning_data/_people_person_nationality/parsed/dev_matrix.json')

In [3]:
train = pd.concat([pos, neg], axis=0).reset_index()

In [4]:
train.head()

,index,source,target,relation_paths,entity_paths,label,target_relation
0,0,/m/06w5j9r,/m/06mkj,"[[for,the,4,to,the, _with, #END], [in,the, Che...","[[/m/06w5j9r, /m/02jx1, /m/06mkj], [/m/06w5j9r...",1,/people/person/nationality
1,1,/m/05b4fq7,/m/09c7w0,"[[received,his,2,from,the, _and,in,2,for,the, ...","[[/m/05b4fq7, /m/02hp6p, /m/04yty0, /m/036nz, ...",1,/people/person/nationality
2,2,/m/07r523,/m/0154j,"[[/people/person/place_of_birth, and,the, who,...","[[/m/07r523, /m/0cf0s, /m/0hzlz, /m/02j9z, /m/...",1,/people/person/nationality
3,3,/m/01gtny,/m/0d060g,"[[defending, _felt,that, _will,play,the, _titl...","[[/m/01gtny, /m/07jl8v, /m/0hn2q, /m/0hmtk, /m...",1,/people/person/nationality
4,4,/m/08c7fr,/m/0chghy,"[[and,Indian, on,a, model,in, the,US, #END], [...","[[/m/08c7fr, /m/0g9_1gq, /m/0h7mwj9, /m/03rk0,...",1,/people/person/nationality


In [5]:
len(train)

15083

In [6]:
train['max_path_len'] = train['relation_paths'].apply(lambda row: np.max([len(path) for path in row]))

In [7]:
train['max_path_len'].max()

16

In [8]:
train['num_paths'] = train['relation_paths'].apply(lambda row: len(row))

In [9]:
train['num_paths'][0]

127

In [10]:
train['num_paths'].max()

947

In [11]:
relations = set(chain.from_iterable(list(chain.from_iterable(train['relation_paths']))))
relations.add('#UNK')
relations.add('#END')

In [12]:
random_embd = RandomEmbeddings(relations, 250, 'rel_embd', '#UNK', tf.initializers.random_normal())

In [13]:
random_embd.get_idx('_felt,that')

12606

In [14]:
random_embd.get_idx('lol this should not work')

6250

In [15]:
train.loc[train['label'] == 0]['num_paths'].sum()

1014917

In [16]:
train.loc[train['label'] == 1]['num_paths'].sum()

785581

In [17]:
dev.head()

,source,target,relation_paths,entity_paths,label,target_relation
0,/m/0fw80h,/m/05b4w,"[[shows,how, or, _Chamber,of,Commerce, _in, _/...","[[/m/0fw80h, /m/017yh, /m/01nsmg, /m/0bfc5q, /...",0,/people/person/nationality
1,/m/04y6lhs,/m/06s_2,"[[is, _from, during, Portugal, _/people/person...","[[/m/04y6lhs, /m/09b6zr, /m/07t31, /m/01crd5, ...",0,/people/person/nationality
2,/m/0437mm,/m/012m_,"[[/people/person/place_of_birth, _/location/lo...","[[/m/0437mm, /m/04ckd4, /m/05qhw, /m/012m_], [...",0,/people/person/nationality
3,/m/0f1cgl,/m/03_3d,"[[_NEW,YORK, over,the,3,in,the, South,Africa,a...","[[/m/0f1cgl, /m/0cv_2, /m/09c7w0, /m/03_3d], [...",1,/people/person/nationality
4,/m/047qmvx,/m/03spz,"[[but,the, Sir, was,chosen,by, from, #END], [b...","[[/m/047qmvx, /m/0cp02r6, /m/0jrc0, /m/07ssc, ...",0,/people/person/nationality


In [19]:
dev.loc[dev['label'] == 1].count()

source             1038
target             1038
relation_paths     1038
entity_paths       1038
label              1038
target_relation    1038
dtype: int64

In [20]:
dev.loc[dev['label'] == 0].count()

source             12466
target             12466
relation_paths     12466
entity_paths       12466
label              12466
target_relation    12466
dtype: int64